In [288]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import math

In [289]:
reader = csv.reader(open("hepatitis_2_csv.csv", "r"))
x = list(reader)
res = np.array(x).astype("float")

In [290]:
training_class_name=res[0:568,19:20];
training_data=res[0:568,0:19];
training_data=np.matrix(training_data)
training_data=training_data.transpose()
training_class_name=np.matrix(training_class_name)
training_class_name=training_class_name.transpose()
print(np.prod(list(training_class_name[0].shape)))
print(training_data.shape)

568
(19, 568)


In [291]:
meanData=training_data.mean()
varianceData=np.var(training_data)
training_data=(training_data-meanData)/varianceData

In [299]:
class Net:
    def __init__(self):
        self.weights=[];
        self.bias=[];
        self.status=[];
        self.a=[[[]]];
        self.z=[[[]]];
        self.da=[[[]]];
        self.dz=[[[]]];
        self.dw=[[[]]];
        self.db=[[[]]];
        self.cost=[];
        self.trainingOutput=[];
        self.correctOutput=[];
        self.size=0;
        self.vw=[];
        self.vb=[];
        self.sw=[];
        self.sb=[];
        self.vwcorrected=[];
        self.vbcorrected=[];
        self.swcorrected=[];
        self.sbcorrected=[];
        
        
    def model(self,layers):
        self.layers=layers
        self.status.append("Model archetecture recieved")
        self.createBrain()
        self.status.append("Model archetecture created")
        return self.layers
    
    def createBrain(self):
        inputshape = self.layers[0].getshape()
        self.weights.append(0.01*np.random.rand(self.layers[1].getparams()["noOfUnits"],inputshape[0]))
        self.bias.append(np.ones([self.layers[1].getparams()["noOfUnits"],inputshape[1]]))
        print("layer 1 has bias shape as {} and weights shape as {}".format(self.bias[0].shape,self.weights[0].shape))
        for i in range(2,len(self.layers)):
            lowerlimit=self.layers[i].getparams()["noOfUnits"]
            upperlimit=self.layers[i-1].getparams()["noOfUnits"]
            self.weights.append(0.01*np.random.rand(lowerlimit,upperlimit))
            self.bias.append(np.ones([lowerlimit,inputshape[1]]))
            print("layer {} has bias shape as {} and weights shape as {}".format(i,self.bias[i-1].shape,self.weights[i-1].shape))
    
    def train(self,traindata,trainoutcome,hyperparams):
        
        numberOfExamples = np.prod(list(training_class_name[0].shape))   
        epochs=hyperparams["epochs"]
        self.epochs=np.asmatrix(np.arange(0,epochs,1))
        democost=0
        
        if(hyperparams["Batch_size"]>0):
            minibatch=True
            size=hyperparams["Batch_size"]
        
            if(minibatch):
                self.bias[0]=np.ones([self.layers[1].getparams()["noOfUnits"],size])
                for i in range(1,len(self.layers)):
                    lowerlimit=self.layers[i].getparams()["noOfUnits"]
                    self.bias[i-1]=np.ones([lowerlimit,size])

                minibatches = self.make_mini_batches(numberOfExamples,traindata,trainoutcome,30)
        else:
            minibatches = [[(traindata),(trainoutcome)]]
            
        for i in range(0,epochs):

            
            for j in range(0,len(minibatches)):                              
                # Select a minibatch
                (traindata, trainclass) = minibatches[j]
                self.size = traindata.shape[1] 
                op = self.forewardProp(numberOfExamples,traindata,trainclass,i,j)
                val = op["val"]
                               
                if(self.layers[1].getparams()["activation"]=="sigmoid"):
                    val=val>=0.499
                elif(self.layers[1].getparams()["activation"]=="tanh" or self.layers[1].getparams()["activation"]=="relu"):
                    val=val>=0
                 
                democost=democost+op["cost"]
                
                self.trainingOutput.append(val)
                self.correctOutput.append(trainoutcome)
                self.status.append("Foreward Propogation for batch {} in epoch {} is completed".format(j,i))
                self.backwardProp(numberOfExamples,trainclass,i,j,hyperparams)
                self.status.append("Backward Propogation for batch {} in epoch {} is completed".format(j,i))
                self.da[i].append([])
                self.a[i].append([])
                self.z[i].append([])
                self.dz[i].append([])
                self.dw[i].append([])
                self.db[i].append([])
                
            self.cost.append(democost)    
            print("cost for epoch {} is {}".format(i,democost))   
            self.status.append("Foreward Propogation for epoch {} is completed".format(i))
            self.status.append("Backward Propogation for epoch {} is completed".format(i))                  
            self.da.append([[]])
            self.dz.append([[]])
            self.dw.append([[]])
            self.db.append([[]])
            self.a.append([[]])
            self.z.append([[]])
            
        #self.trainingOutput=np.asarray(self.trainingOutput)
        #self.correctOutput=np.asarray(self.correctOutput)
        #self.dz=np.asarray(self.dz)
        #self.dw=np.asarray(self.dw)
        #self.db=np.asarray(self.db)
        #self.da=np.asarray(self.da)

    
    def forewardProp(self,m,traindata,trainoutcome,iterno,batch):
        self.a[iterno][batch].append(traindata)
        z=self.weights[0]*traindata+self.bias[0][:,np.arange(0,self.size)]
        self.z[iterno][batch].append(z)
        
        if(self.layers[1].getparams()["activation"]=="sigmoid"):
            aval=self.sigmoid(z)
            self.a[iterno][batch].append(aval)
        elif(self.layers[1].getparams()["activation"]=="tanh"):
            aval=self.tanh(z)
            self.a[iterno][batch].append(aval)
        elif(self.layers[1].getparams()["activation"]=="relu"):
            aval=self.relu(z)
            self.a[iterno][batch].append(aval)
        #print(aval.shape)
        
        
        for i in range(2,len(self.layers)):
            #print(self.weights[i-1].shape,self.a[iterno][batch][i-1].shape,self.bias[i-1].shape)
            z=(self.weights[i-1]*self.a[iterno][batch][i-1])+self.bias[i-1][:,np.arange(0,self.size)]
            self.z[iterno][batch].append(z)
            #print(z)
            if(self.layers[1].getparams()["activation"]=="sigmoid"):
                aval=self.sigmoid(z)
                self.a[iterno][batch].append(aval)
            elif(self.layers[1].getparams()["activation"]=="tanh"):
                aval=self.tanh(z)
                self.a[iterno][batch].append(aval)
            elif(self.layers[1].getparams()["activation"]=="relu"):
                aval=self.relu(z)
                self.a[iterno][batch].append(aval)
            #print("self.weights = {} trainingdata = {} bias = {} z = {}".format(self.weights[i-1].shape,self.a[iterno][batch][i-1].shape,self.bias[i-1].shape,z.shape))
        #print("a = {}".format(self.a[iterno][batch][len(self.layers)-1]))
        #print(self.z[iterno][batch][len(self.layers)-2])
        
        cost=self.costfunc(m,traindata,trainoutcome,self.a[iterno][batch][i])                   
        returnval = {"val":self.a[iterno][batch][len(self.layers)-1],"cost":cost}
            
        return returnval
     
    def costfunc(self,m,traindata,trainoutcome,res):
        #print(np.multiply(1-trainoutcome,np.log(1-res)))
        A1=np.multiply(trainoutcome,np.log(res))
        A2=np.multiply((1-trainoutcome),np.log((1-res)))
        cost=A1+A2
        cost=np.sum(cost)
        cost=(-1/m)*cost
        return cost
    
    def backwardProp(self,m,trainoutcome,iterno,batch,hyperparams):
        self.optimizationinitialize()
        AL=self.a[iterno][batch][len(self.layers)-1]
        dAL= - ( np.divide(trainoutcome,AL) - np.divide(1-trainoutcome,1-AL) )
        self.da[iterno][batch].append(dAL);
        for i in range(0,len(self.layers)-1): 
            sigmoidprime=self.sigmoid_Prime(self.z[iterno][batch][len(self.layers)-2-i])
            dz = np.multiply(dAL,sigmoidprime)
            self.dz[iterno][batch].append(dz)
            dw = (1/self.size) * ( dz * ( self.a[iterno][batch][len(self.layers)-2-i].transpose() ) )
            #print(dw)
            self.dw[iterno][batch].append(dw)
            #dz = np.array(dz)
            db = (1/self.size)*np.sum(dz,axis=1)
            #print(db)
            #db = np.asmatrix(db)
            #dz = np.asmatrix(dz)
            self.db[iterno][batch].append(db)
            dAL=(self.weights[len(self.layers)-2-i].transpose())*dz
            self.da[iterno][batch].append(dAL)
            # Updating weights
            if(hyperparams["optimizer"]=="momentum"):
                self.updateMomentum(hyperparams["beta"],hyperparams["LearningRate"],dw,db,i)
            elif(hyperparams["optimizer"]=="adam"):
                self.updateAdam(hyperparams["beta1"],hyperparams["beta2"],hyperparams["LearningRate"],dw,db,i,batch+1,hyperparams["epsilon"])
            elif(hyperparams["optimizer"]=="gd"):
                self.updateGD(hyperparams["LearningRate"],dw,db,i)
            #print("bias are {}".format(self.bias[len(self.layers)-2]))
            
    def optimizationinitialize(self):
        for i in range(0,len(self.layers)-1):
            self.vw.append(np.zeros(list(self.weights[i].shape)))
            self.vb.append(np.zeros(list(self.bias[i].shape)))
            self.sw.append(np.zeros(list(self.weights[i].shape)))
            self.sb.append(np.zeros(list(self.bias[i].shape)))
            self.vwcorrected.append(np.zeros(list(self.weights[i].shape)))
            self.vbcorrected.append(np.zeros(list(self.bias[i].shape)))
            self.swcorrected.append(np.zeros(list(self.weights[i].shape)))
            self.sbcorrected.append(np.zeros(list(self.bias[i].shape)))
            
        
    def updateGD(self,learningRate,dw,db,i):
        self.weights[len(self.layers)-2-i] = self.weights[len(self.layers)-2-i] - learningRate*dw
        self.bias[len(self.layers)-2-i] = self.bias[len(self.layers)-2-i] - learningRate*db
        
    def updateMomentum(self,beta,learningRate,dw,db,i):
        self.vw[len(self.layers)-2-i] = beta*self.vw[len(self.layers)-2-i] + (1-beta)*dw
        self.vb[len(self.layers)-2-i] = beta*self.vb[len(self.layers)-2-i] + (1-beta)*db
        
        self.weights[len(self.layers)-2-i] = self.weights[len(self.layers)-2-i] - learningRate*self.vw[len(self.layers)-2-i]
        self.bias[len(self.layers)-2-i] = self.bias[len(self.layers)-2-i] - learningRate*self.vb[len(self.layers)-2-i]
    
    def updateAdam(self,beta1,beta2,learningRate,dw,db,i,batch,epsilon):
        #print("beta1 = {} beta2 = {} learningRate = {} dw ={} db = {} i = {} batch ={} epsilon = {}".format(beta1,beta2,learningRate,dw,db,i,batch,epsilon))
       
        self.vw[len(self.layers)-2-i] = beta1*self.vw[len(self.layers)-2-i] + (1-beta1)*dw
        self.vb[len(self.layers)-2-i] = beta1*self.vb[len(self.layers)-2-i] + (1-beta1)*db
        
        #print("beta = {} batch = {} val = {}".format(beta1,batch,1 - np.power(beta1,batch)))
        
        vw_corrected = self.vw[len(self.layers)-2-i] / (1 - np.power(beta1,batch) )
        vb_corrected = self.vb[len(self.layers)-2-i] / (1 - np.power(beta1,batch) )
        
        self.vwcorrected[len(self.layers)-2-i]=vw_corrected
        self.vbcorrected[len(self.layers)-2-i]=vb_corrected
        
        self.sw[len(self.layers)-2-i] = beta2*self.sw[len(self.layers)-2-i] + (1-beta2)*np.power(dw,2)
        self.sb[len(self.layers)-2-i] = beta2*self.sb[len(self.layers)-2-i] + (1-beta2)*np.power(db,2)
        
        sw_corrected = self.sw[len(self.layers)-2-i] / (1 - np.power(beta2,batch) )
        sb_corrected = self.sb[len(self.layers)-2-i] / (1 - np.power(beta2,batch) )
        
        self.swcorrected[len(self.layers)-2-i]=sw_corrected
        self.sbcorrected[len(self.layers)-2-i]=sb_corrected
        
        #print("val = ",(vw_corrected / np.sqrt(sw_corrected + epsilon)))
        #print()
        #print("first time  =",self.weights[len(self.layers)-2-i])
        self.weights[len(self.layers)-2-i] = self.weights[len(self.layers)-2-i] - learningRate * (vw_corrected / np.sqrt(sw_corrected + epsilon))
        self.bias[len(self.layers)-2-i] = self.bias[len(self.layers)-2-i] - learningRate * (vb_corrected / np.sqrt(sb_corrected + epsilon))
        
        
        #print("second time = ",self.weights[len(self.layers)-2-i])
        
    
    def make_mini_batches(self,m,training_data,training_class_names,batch_Size):
        mini_batches=[]
        new_training_data=training_data
        new_training_class_names=training_class_names
        # Randomize The Training Data
        #permutation = list(np.random.permutation(m))
        #new_training_data = training_data[:,permutation]
        #new_training_class_names=training_class_names[:, permutation].reshape((1,m))
        # Get Number Of Batches
        num_minibatches = math.floor(m/batch_Size)
        # Split The Training_Data Into Batches
        for i in range(0,num_minibatches):
            training_batch = new_training_data[:,batch_Size*i:batch_Size *(i+1)]
            training_class_batch = new_training_class_names[:,batch_Size*i:batch_Size *(i+1)]
            mini_batch = (training_batch, training_class_batch)
            mini_batches.append(mini_batch)
            
        # Last Batch If Exists
        if m % batch_Size != 0:
            training_batch = new_training_data[:,batch_Size*(i+1):m]
            training_class_batch = new_training_class_names[:,batch_Size*(i+1):m]
            ### END CODE HERE ###
            mini_batch = (training_batch, training_class_batch)
            mini_batches.append(mini_batch)
        
        mini_batches = np.asarray(mini_batches)
            
        return mini_batches
        
    def sigmoid_Prime(self,value):
        part1=self.sigmoid(value)
        part2=1-(self.sigmoid(value))
        return np.multiply( part1 , part2 )
    
    def tanh_Prime(self,value):
        return 1-(self.tanh(value)*self.tanh(value))
    
    def relu_Prime(self,value):
        return value>=0
        
    def results(self):
        trainingOutput=np.asarray(self.trainingOutput)
        correctOutput=np.asarray(self.correctOutput)
        print(trainingOutput.shape,correctOutput.shape)
        #print(self.correctOutput.shape,self.trainingOutput.shape)
        ls=correctOutput - trainingOutput
        loss=np.sum(ls,axis=1)
        loss=loss/len(loss)
        ac=np.sum(correctOutput==trainingOutput,axis=1)
        accuracy=ac/len(self.correctOutput)
        plt.subplot(1,2,1)
        plt.plot(self.epochs.T,accuracy)
        plt.title("Training Accuracy")
        plt.xlabel("epochs")
        plt.ylabel("Accuracy")
        plt.subplot(1,2,2)
        plt.plot(self.epochs.T,loss)
        plt.title("Training Loss")
        plt.xlabel("epochs")
        plt.ylabel("Loss")
        #print("Accuracy : {}".format(accuracy[len(accuracy)-1]))
        #print("loss : {}".format(loss[len(loss)-1]))
    
    def sigmoid(self,X):
        return 1/(1+np.exp(-X));

    def tanh(self,X):
        return np.tanh(X);

    def relu(self,X):
        return np.maximum(0, X);

    def getActivation(self):
        # plot the activation function using matplotlib library
        cur_axes = plt.gca()
        # to remove the x axis
        cur_axes.axes.get_xaxis().set_visible(False)
        rangex=np.linspace(-10, 10, 100)

        if(self.activation=="sigmoid"):
            plt.plot(rangex,self.sigmoid(rangex))
        elif(self.activation=="tanh"):
            plt.plot(rangex,self.tanh(rangex))
        elif(self.activation=="relu"):
            plt.plot(rangex,self.relu(rangex))
            plt.title("activation function = "+self.activation)
        
    class Layers:
        
        class InputLayer:
            
            def __init__(self,shape):
                self.shape=shape
                
            def getshape(self):
                return np.asarray(self.shape).transpose()
        
        class DenseLayer:
            
            def __init__(self,noOfUnits,activation):
                self.params={"activation":activation,"shape":(noOfUnits,1),"noOfUnits":noOfUnits}
                
            def getparams(self):
                return self.params

In [300]:
my_brain = Net()
l1=Net.Layers.InputLayer(training_data.shape)
l2=Net.Layers.DenseLayer(15 , activation="sigmoid")
l3=Net.Layers.DenseLayer(10 , activation="sigmoid")
l4=Net.Layers.DenseLayer(5 , activation="sigmoid")
l5=Net.Layers.DenseLayer(4 , activation="sigmoid")
l6=Net.Layers.DenseLayer(1 , activation="sigmoid")
archetecture = my_brain.model(
    layers=[l1,l2,l3,l4,l5,l6]
)

layer 1 has bias shape as (15, 568) and weights shape as (15, 19)
layer 2 has bias shape as (10, 568) and weights shape as (10, 15)
layer 3 has bias shape as (5, 568) and weights shape as (5, 10)
layer 4 has bias shape as (4, 568) and weights shape as (4, 5)
layer 5 has bias shape as (1, 568) and weights shape as (1, 4)


In [301]:
settings={"LearningRate":0.001,"Batch_size":30,"optimizer":"adam","beta1":0.9,"beta2":0.999,"epochs":10,"epsilon": 1e-8}

In [302]:
my_brain.train(training_data,training_class_name,settings)

cost for epoch 0 is 0.49437735844637504
cost for epoch 1 is 0.9883672818439576
cost for epoch 2 is 1.481978648014717
cost for epoch 3 is 1.9752201113968277
cost for epoch 4 is 2.4681001092772075
cost for epoch 5 is 2.96062686783112
cost for epoch 6 is 3.4528084079754238
cost for epoch 7 is 3.94465255104178
cost for epoch 8 is 4.43616692427591
cost for epoch 9 is 4.927358966168791


In [296]:
result = my_brain.make_mini_batches(568,training_data,training_class_name,30)

In [297]:
print(result[18][1].shape)

(1, 28)


In [298]:
results=my_brain.results()

ValueError: could not broadcast input array from shape (30) into shape (1)

In [ ]:
a=np.array([[1,2,3,4,5],[4,5,6,7,8],[7,8,9,1,2],[11,21,31,41,15],[4,5,6,7,8],[7,8,9,1,2],[12,22,32,42,52],[4,5,6,7,8],[7,8,9,1,2]])
print(a)

In [ ]:
permutation = list(np.random.permutation(len(a)))
print(permutation)
new_a = a[permutation,:]
print(new_a)

In [ ]:
abc=[[[[0],[1],[2]]]]

In [ ]:
print(abc[0][0][1])

In [ ]:
np.arange(0,5)

In [ ]:
print(np.floor(560/30))